# Python私房手册-类型提示

在Python3.5版本以后，就可以使用类型提示了，几个主要的官方文档如下：
- [PEP484: 类型提示，包含参数注释以及在#注释中写变量注释的语法](https://www.python.org/dev/peps/pep-0484/)
- [PEP526: 变量注释，直接在代码中写变量注释的语法](https://www.python.org/dev/peps/pep-0526/)


相关教程：
- [Python Type Checking](https://realpython.com/python-type-checking/#type-theory)
- [Python类型检查（中文）](https://blog.csdn.net/u011331731/article/details/108354536)
- [Sorry, you’re just not my type](https://blog.labdigital.nl/sorry-youre-just-not-my-type-e1b8934c1013)

## 参数注释

### TypeVar

#### TypeVar的bound参数

```python
ST = TypeVar('ST', bound=Sized)


def longer(x: ST, y: ST) -> ST:
    if len(x) > len(y):
        return x
    else:
        return y
    
    
longer([1], [1, 2])  # ok, return type List[int]
longer({1}, {1, 2})  # ok, return type Set[int]
longer([1], {1, 2})  # PEP484：ok, return type Collection[int]，但是pycharm2021.3提示错误
```
这是PEP484的一个例子，但是`longer([1], {1, 2})`在pycharm2021.3版本中提示错误，不知道在后续的PEP中进行了修改，还是pycharm不完善，不过pycharm提示错误才更符合我的理解。因为`bound`只是提供一个类型的上限，但是`ST`应该始终代表同一类型，不然的话，`ST`和普通的`UNION`就没啥区别了。

## 变量注释

### 基本语法

在`#`注释中写变量提示有很多不足，因此在PEP526中规定了直接在代码中书写变量提示的语法，类型提示可以直接添加到赋值语句中或者单个的表达式中，如下，两种写法都是可以的：
```python
# 第一种写法
my_var: int
my_var = 5

# 第二种写法
other_var: int = 5
```
虽然两种写法效果都可以，但是如果有分支的情况下，第一种写法更好：
```python
my_var: bool

if 2 + 2 == 4:
    my_var = True
else:
    my_var = False
```
注意，当元组打包的时候，语法允许变量提示，但是解包时，是不允许的：
```python
from typing import Tuple

t: Tuple[int, ...] = (1, 2, 3)  # 省略号表示其它所有元素都是int类型
t: Tuple[int, ...] = 1, 2, 3  # 3.8版本以后也可以这样写
```
但是解包是不能直接写类型提示的：
```python
a:int, b:int, c:int = (1, 2, 3)  # 不能这样写，提示语法错误
```
只能够每个变量先注释：
```python
header: str
kind: int
body: Optional[List[str]]  # 表示body为str构成的列表或者为None
header, kind, body = message
```

### 全局和局部的变量注释

注意，当全局变量注释时，会产生一个未绑定的变量：
```python
a: int
print(a)  # 抛出NameError，变量未定义
```
如果是在函数内被定义，那么会产生一个局部的未绑定的变量：
```python
a = 42

def func():
    a: int
    print(a) 

func()  # UnboundLocalError: local variable 'a' referenced before assignment
```

### 类和实例的变量注释

类型注释还可以用于注释类主体和方法中的类和实例变量。特别是，无值表示法`a: int`允许对应该在`__init__`或`__new__`中初始化的实例变量进行注释。建议的语法如下所示:
```python
class BasicStarship:
    captain: str = 'Picard'               # instance variable with default
    damage: int                       # instance variable without default
    stats: ClassVar[Dict[str, int]] = {}      # class variable
```
虽然`caption`是类属性，但是其目的是所有实例变量设置一个默认值，所以在`__init__`中还需要定义一个`captain`的实例变量。但是，如果像下面这样写：
```python
class BasicStarship:
    captain: str = 42  # captain只会检查实例属性是否是str，不会检查类属性
```
此时是不会提示错误的，说实话，这挺容易让人产生误解的。而真正的类变量不应该被实例变量覆盖，因此需要使用`ClassVar`注释，此时如果意外设置了实例的`stats`属性，编辑器会提示错误。

也可以直接对实例变量进行注释：
```python
class BasicStarship:
    def __init__(self, captain):
        self.captain: str = captain
```
注意一点，ClassVar不能包含任何类型变量

## 不变、协变和逆变